In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2

In [2]:
import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.utils import get_model, visualize_2motions, generate_src_mask, uniform, cosine_schedule
from utils.motion_process import recover_from_ric

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 't2m'
args.down_t = 2

train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

dataset_opt_path = 'checkpoints/kit/Comp_v6_KLD005/opt.txt' if args.dataname == 'kit' else 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt


100%|██████████| 1460/1460 [00:00<00:00, 1806.67it/s]


Pointer Pointing at 0
Reading checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 28) Completed!!


In [4]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

In [5]:
args.nb_code = 512 # 8192 # 
args.code_dim = 512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None
args.quantizer = 'ema_reset'
args.mu = 0.99

args.vq_name = 'VQVAE'
args.vq_dir= os.path.join("./dataset/KIT-ML" if args.dataname == 'kit' else "./dataset/HumanML3D", f'{args.vq_name}')
os.makedirs(args.vq_dir, exist_ok = True)


net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate)
args.embed_dim_gpt = 1024
args.clip_dim = 512
args.block_size = 51
args.num_layers = 9
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4
trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code, 
                                            embed_dim=args.embed_dim_gpt, 
                                            clip_dim=args.clip_dim, 
                                            block_size=args.block_size, 
                                            num_layers=args.num_layers, 
                                            n_head=args.n_head_gpt, 
                                            drop_out_rate=args.drop_out_rate, 
                                            fc_rate=args.ff_rate)
args.resume_pth = '/home/epinyoan/git/MaskText2Motion/T2M-GPT/pretrained/VQVAE/net_best_fid.pth'
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
net.eval()
net.cuda()

args.resume_trans = '/home/epinyoan/git/MaskText2Motion/T2M-GPT/pretrained/VQTransformer_corruption05/net_best_fid.pth'
ckpt = torch.load(args.resume_trans, map_location='cpu')
trans_encoder.load_state_dict(ckpt['trans'], strict=True)
trans_encoder.train()
trans_encoder.cuda()
''

''

### VQVAE encoder index precomputed

In [8]:
for batch in train_loader_token:
    pose, name = batch
    bs, seq = pose.shape[0], pose.shape[1]

    pose = pose.cuda().float() # bs, nb_joints, joints_dim, seq_len
    target = net.encode(pose)
    target = target.cpu().numpy()
    np.save(pjoin(args.vq_dir, name[0] +'.npy'), target)

### Train

In [9]:
args.batch_size = 2
num_workers = 8
args.vq_name = 'VQVAE'
train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, args.vq_name, unit_length=2**args.down_t, num_workers=num_workers)
train_loader_iter = dataset_TM_train.cycle(train_loader)

100%|██████████| 23384/23384 [00:06<00:00, 3355.72it/s]


In [ ]:
######## count used indices
# all_indices = None
# for batch in train_loader:
#     clip_text, m_tokens, m_tokens_len = batch
#     if all_indices is None:
#         all_indices = m_tokens
#     else:
#         all_indices = torch.cat([all_indices, m_tokens])
# a = all_indices.view(-1)
# condition = a < 512
# c = torch.bincount(a[condition])
# (c==0).sum()

In [10]:
args.pkeep = .5
batch = next(train_loader_iter)
clip_text, m_tokens, m_tokens_len = batch
m_tokens, m_tokens_len = m_tokens.cuda(), m_tokens_len.cuda()
bs = m_tokens.shape[0]
target = m_tokens    # (bs, 26)
target = target.cuda()

text = clip.tokenize(clip_text, truncate=True).cuda()
    
feat_clip_text = clip_model.encode_text(text).float()

input_index = target[:,:-1]

if args.pkeep == -1:
    proba = np.random.rand(1)[0]
    mask = torch.bernoulli(proba * torch.ones(input_index.shape,
                                                        device=input_index.device))
else:
    mask = torch.bernoulli(args.pkeep * torch.ones(input_index.shape,
                                                        device=input_index.device))
mask = mask.round().to(dtype=torch.int64)
r_indices = torch.randint_like(input_index, args.nb_code)
a_indices = mask*input_index+(1-mask)*r_indices

cls_pred = trans_encoder(a_indices, feat_clip_text)
cls_pred = cls_pred.contiguous()
cls_pred.shape

torch.Size([2, 51, 513])

In [11]:
target

tensor([[297, 343, 501, 215, 338, 501, 215, 400, 400, 178, 400, 400, 400, 266,
         139, 139, 338, 397, 215, 282, 105, 459, 459, 459, 459, 459, 459, 459,
         501,  25, 305,   1, 124, 139, 512, 513, 513, 513, 513, 513, 513, 513,
         513, 513, 513, 513, 513, 513, 513, 513, 513],
        [511, 102, 319, 458,  66,  54, 125,   0, 362, 509, 511, 470,  99, 107,
          29, 184, 468, 107,  29, 319, 308, 402, 290,   0, 479,  54, 467, 147,
         311, 492, 342, 512, 513, 513, 513, 513, 513, 513, 513, 513, 513, 513,
         513, 513, 513, 513, 513, 513, 513, 513, 513]], device='cuda:0')

In [19]:
args.if_maxtest = False

right_num = 0
loss_ce = torch.nn.CrossEntropyLoss()
loss_cls = 0.0
for i in range(bs):
    # loss function     (26), (26, 513)
    loss_cls += loss_ce(cls_pred[i][:m_tokens_len[i] + 1], target[i][:m_tokens_len[i] + 1]) / bs

    # Accuracy
    probs = torch.softmax(cls_pred[i][:m_tokens_len[i] + 1], dim=-1)

    if args.if_maxtest:
        _, cls_pred_index = torch.max(probs, dim=-1)
    else:
        dist = Categorical(probs)
        cls_pred_index = dist.sample()
    right_num += (cls_pred_index.flatten(0) == target[i][:m_tokens_len[i] + 1].flatten(0)).sum().item()

### Eval

In [5]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

100%|██████████| 1460/1460 [00:00<00:00, 2451.30it/s]

Pointer Pointing at 0


In [6]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

test
./Test/FID fid nb_iter


In [7]:
args.out_dir = './'
pred_pose_eval, pose, m_length, clip_text, best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, writer, logger = \
    eval_trans.evaluation_transformer(args.out_dir, 
        val_loader, 
        net, 
        trans_encoder, 
        logger, 
        writer, 
        0, 
        best_fid=1000, 
        best_iter=0, 
        best_div=100, 
        best_top1=0, 
        best_top2=0, 
        best_top3=0, 
        best_matching=100, 
        clip_model=clip_model, 
        eval_wrapper=eval_wrapper)

100%|██████████| 47/47 [08:10<00:00, 10.44s/it]


--> 	 Eva. Iter 0 :, FID. 0.1042, Diversity Real. 9.2803, Diversity. 9.2775, R_precision_real. [0.53058511 0.7081117  0.79587766], R_precision. [0.49335106 0.67952128 0.78125   ], matching_score_real. 2.853442374696123, matching_score_pred. 3.085210272606383
./Test/FID 0.1042081499776657 0
./Test/Diversity 9.277454 0
./Test/top1 0.4933510638297872 0
./Test/top2 0.6795212765957447 0
./Test/top3 0.78125 0
./Test/matching_score 3.085210272606383 0
--> --> 	 FID Improved from 1000.00000 to 0.10421 !!!
--> --> 	 matching_score Improved from 100.00000 to 3.08521 !!!
--> --> 	 Diversity Improved from 100.00000 to 9.27745 !!!
--> --> 	 Top1 Improved from 0.0000 to 0.4934 !!!
--> --> 	 Top2 Improved from 0.0000 to 0.6795 !!!
--> --> 	 Top3 Improved from 0.0000 to 0.7812 !!!


### Visualize Prediction

In [30]:
batch = next(iter(val_loader))
word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name = batch
k=10
text = clip.tokenize(clip_text, truncate=True).cuda()
feat_clip_text = clip_model.encode_text(text).float()
index_motion = trans_encoder.sample(feat_clip_text[k:k+1], False)
index_motion

tensor([[256,  26, 102, 231, 221, 173, 442, 237, 174,  89,   2,  10,  43, 287,
         131, 271, 213, 125, 270, 148, 135,  57, 333, 504, 184, 383,  30, 504,
         190, 432,   2, 263, 173, 395,   5, 229, 372, 386, 470,  15,  99, 389,
         344, 270]], device='cuda:0')

In [31]:
pos_one_hots.shape
target = net.encode(pose.cuda().float())
target[k]

tensor([382,  25, 231,  46, 128,  78, 147, 372, 156, 468, 399, 266, 128, 128,
        147,  37,  43, 238, 131, 140, 111,   2, 140, 217, 287, 174, 128, 361,
        372, 431, 399, 266, 467,  15, 287, 465,  54,  35,   2,  39, 494, 419,
        511,  99, 390, 325, 386, 309, 192], device='cuda:0')

In [32]:
bs, seq = pose.shape[:2]
pred_pose_eval = torch.zeros((bs, seq, pose.shape[-1])).cuda()
pred_pose = net.forward_decoder(index_motion)
cur_len = pred_pose.shape[1]
pred_pose_eval[k:k+1, :cur_len] = pred_pose[:, :seq]

In [33]:
x = pose[k].detach().cpu().numpy()
y = pred_pose_eval[k].detach().cpu().numpy()
l = m_length[k]
caption = clip_text[k]
visualize_2motions(x, val_loader.dataset.std, val_loader.dataset.mean, args.dataname, l, y, save_path=None)